In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle
from rocky.model_selection.TriangleTimeSeriesSplit import TriangleTimeSeriesSplit
import pandas as pd
import numpy as np

In [2]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Create a Rocky object
rky = rocky()

# add triangle
# rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
# rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.


In [4]:
cv = TriangleTimeSeriesSplit(triangle=rky.icrfs.tri, model_type='loglinear', loglinear_grid=None)
# dir(cv)

In [5]:
# set hyperparameters
rky.icrfs.SetHyperparameters(alpha=0.025, l1_ratio=0.1)
# rky.icrfs.TuneFitHyperparameters()

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/4.4596 2/1.7314 3/1.3019 4/0.7492 5/0.5390 6/0.3340 7/0.2204 8/0.1332 9/0.0891 10/0.0665 11/0.0523 12/0.0426 13/0.0282 14/0.0185 15/0.0122 16/0.0081 

In [6]:
rky.icrfs.LassoFeatureImportance()

,0.0000,0.0025,0.0050,0.0075,0.0100,0.0125,0.0150,0.0175,0.0200,0.0225,...,0.3600,0.3625,0.3650,0.3675,0.3700,0.3725,0.3750,0.3775,0.3800,0.3825
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.486238,0.368980,0.314178,0.278527,0.243268,0.207449,0.175060,0.148351,0.119196,0.088360,...,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2,0.361121,0.179751,0.119731,0.084945,0.051777,0.019481,0.000000,0.000000,0.000000,0.000000,...,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
3,0.181188,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.00000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
4,0.292698,0.043290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.364919,0.056849,0.000631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,0.468509,0.122080,0.062407,0.013297,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,0.599227,0.197683,0.131638,0.067054,0.012985,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,0.269453,-0.037155,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,0.070475,-0.179899,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [7]:
rky.icrfs.tri.ult()

Accident Period
2001-01-01    3901463.0
2002-01-01    5433719.0
2003-01-01    5378826.0
2004-01-01    5297906.0
2005-01-01    4858200.0
2006-01-01    5111171.0
2007-01-01    5660771.0
2008-01-01    6784799.0
2009-01-01    5642266.0
2010-01-01    4969825.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [8]:
rky.icrfs.Ultimate()

accident_period
2001-01-01    5186334.0
2002-01-01    6777235.0
2003-01-01    6344888.0
2004-01-01    6282395.0
2005-01-01    6372506.0
2006-01-01    6415656.0
2007-01-01    7017375.0
2008-01-01    7169284.0
2009-01-01    6357067.0
2010-01-01    6279795.0
Name: Log-linear Ultimate, dtype: float64

In [9]:
rky.icrfs.TrendPlot()

In [10]:
rky.icrfs.FitPlot()

min: -2.131897483044165, max: 1.1810103933940868


In [18]:
rky.icrfs.ResidualPlot()